In [1]:
import os
import src.utils as utils
import src.Parser.verilog_parser as verilog_parser
from src.Netlist.AST import AST

In [2]:
# path="./docs"
# utils.del_dir_files(path)

In [3]:
top="fsm_0_obf"
pathin="input_files/ASSURE_LOCKED/design1/design1_netlist.v"

# top="fsm"
# pathin="input_files/ASSURE_LOCKED/design1/oracle1_netlist.v"

# utils.clean_dir("./tmp")

# obj=AST(file_path=pathin,rw="w",flag="v",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design

locked = AST(file_path=f"./output_files/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format

Reading LL file
	 Loading json file
	 Done Loading json file
	 Loading Top module in AST
	 Done Loading Top module fsm_0_obf in AST
	 Loading module data in AST
		 Loading module fsm_0_obf in AST
		 Done Loading module fsm_0_obf in AST
	 Done Loading module data in AST
	 Loading gate level data in AST
	 Done Loading gate level data in AST
Done Reading LL file


In [4]:
gate_mapping_vlib,gates_vlib,FF_vlib=locked.gate_mapping_vlib,locked.gates_vlib,locked.FF_vlib

In [5]:
locked.top_module.gates.keys(),locked.top_module.FF_tech.keys()

(dict_keys(['NAND', 'NOT', 'NOR', 'BUF', 'AND', 'OR', 'XOR', 'XNOR']),
 dict_keys(['DFFSNQ_X1', 'DFFRNQ_X1']))

In [6]:
# tech_map={i:i for i in list(locked.top_module.gates,locked.top_module.FF_tech)}

In [7]:
# for gate, gate_list in gate_mapping_vlib.items():
#   print(gate, gate_list)

In [90]:
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/65GP_STDCELL_verilog/tcbn65gplus.v"
path= "vlib/mycells.v"
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules/Train_add_mul_sub_64_bit_Syn_65nm.v"
with open(path,"r") as f:
  module_txt=f.read()
gate_mapping_vlib2,gates_vlib2,FF_vlib2=verilog_parser.extract_modules_def(module_txt)

extract_modules_def took 0.001953125 seconds to complete its execution.


In [55]:
def match_FF(FF1,FF2):
  FF1_I=FF1["inputs"]
  FF2_I=FF2["inputs"].copy()
  if(len(FF1_I)!=len(FF2_I)):
    return False
  # print(FF1_I,FF2_I)
  count=0
  for i in FF1_I:
    for j in FF2_I:
      if(utils.det_FF_node(i)==utils.det_FF_node(j)):
        # print(i,j)
        count+=1
        # FF1_I.remove(i)
        FF2_I.remove(j)
        break
  
  return count==len(FF1_I)

def gen_FF_techmap(FF_target,FF_source):
  # tech_map={target:[source]}
  tech_map={}
  for j in FF_source:
    for i in FF_target:
      if(match_FF(FF_target[i],FF_source[j])):
        if(i not in tech_map):
          tech_map[j]=i
          break
        # else:
        #   tech_map[i].append(j)
  return tech_map


In [57]:
# tech_map=gen_FF_techmap(FF_vlib2,FF_vlib)
# print(FF_to_txt_techmap(locked.top_module.FF_tech,FF_vlib2,tech_map))

In [64]:
def gen_tech_map(gate_mapping_vlib,FF_vlib,gates_vlib,gate_mapping_vlib2,FF_vlib2,gates_vlib2):
  gate_mapping_vlib=gate_mapping_vlib.copy()
  gate_mapping_vlib.pop("FF")
  tech_map=gen_FF_techmap(FF_vlib2,FF_vlib)

  for i in gate_mapping_vlib:
    for org_gate in gate_mapping_vlib[i]:
      for target_gate in gate_mapping_vlib2[i]:
        if(len(gates_vlib[org_gate]['inputs'])==len(gates_vlib2[target_gate]['inputs'])):
          tech_map[org_gate]=target_gate
          break
  return tech_map



In [66]:
gate_mapping_vlib.keys()

dict_keys(['FF', 'OR', 'BUF', 'NOT', 'AND', 'XOR', 'NOR', 'NAND', 'XNOR'])

In [65]:
tech_map=gen_tech_map(gate_mapping_vlib,FF_vlib,gates_vlib,gate_mapping_vlib2,FF_vlib2,gates_vlib2)

In [70]:
locked.top_module.gates["NOT"].items()

dict_items([('INV_X1', {'NOT_U1101': {'inputs': ['n3317'], 'outputs': 'n3376'}, 'NOT_U1160': {'inputs': ['n1729'], 'outputs': 'n1556'}, 'NOT_U1220': {'inputs': ['n1382'], 'outputs': 'n1385'}, 'NOT_U1239': {'inputs': ['n1389'], 'outputs': 'n1096'}, 'NOT_U1244': {'inputs': ['n1254'], 'outputs': 'n1252'}, 'NOT_U1247': {'inputs': ['n1118'], 'outputs': 'n1352'}, 'NOT_U1279': {'inputs': ['n1122'], 'outputs': 'n1121'}, 'NOT_U1285': {'inputs': ['n1732'], 'outputs': 'n1183'}, 'NOT_U1288': {'inputs': ['n1702'], 'outputs': 'n2225'}, 'NOT_U1336': {'inputs': ['n1456'], 'outputs': 'n1751'}, 'NOT_U1412': {'inputs': ['n1641'], 'outputs': 'n1631'}, 'NOT_U1423': {'inputs': ['n4166'], 'outputs': 'n3720'}, 'NOT_U1425': {'inputs': ['n3111'], 'outputs': 'n1311'}, 'NOT_U1430': {'inputs': ['n1670'], 'outputs': 'n1300'}, 'NOT_U1432': {'inputs': ['n1662'], 'outputs': 'n1260'}, 'NOT_U1434': {'inputs': ['n1398'], 'outputs': 'n1297'}, 'NOT_U1437': {'inputs': ['n1298'], 'outputs': 'n1397'}, 'NOT_U1439': {'inputs': 

In [114]:
def gates_to_txt_techmap(gates_tech,gates_vlib_2,tech_map):
  txt=""
  for _,logic_gates in gates_tech.items():
    for logic_gate,gate_inits in logic_gates.items():
      node_name=tech_map[logic_gate]
      port=gates_vlib_2[node_name]['port']
      for init_name in gate_inits:
        tmp={}
        for mIO,mNode in zip(gates_vlib_2[node_name]["inputs"],gate_inits[init_name]['inputs']):
          tmp[mIO]=mNode
        tmp[gates_vlib_2[node_name]["outputs"][0]]=gate_inits[init_name]['outputs']
        txt+=f"{node_name} {init_name} {port.format(**tmp)}\n"
  return txt

In [115]:
txt=gates_to_txt_techmap(locked.top_module.gates,gates_vlib2,tech_map)
txt+=FF_to_txt_techmap(FF_tech,FF_vlib_2,tech_map)

In [ ]:
def FF_to_txt_techmap(FF_tech,FF_vlib_2,tech_map):
  # tech_map={target:[source]}
  txt=""
  for FF_tech_i in FF_tech:
    node_name=tech_map[FF_tech_i]
    port=FF_vlib_2[node_name]['port']
    for init_name in FF_tech[FF_tech_i]:
      tmp={}
      for mIO in FF_vlib_2[node_name]["port_list"]:
        tmp[mIO]=FF_tech[FF_tech_i][init_name][utils.det_FF_node(mIO)]
      txt+= f"{node_name} {init_name} {port.format(**tmp)}\n"
  return txt

In [ ]:
# path_dir=r"/mnt/d/alis_files/LAPTOP/alis_files/university_files/GDRIVE/Hardware security FYP/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules"
# paths=[os.path.join(path_dir,i) for i in os.listdir(path_dir) if(i[:5]=="Train")]

# import re
# def analyze_init(paths):
#   items={}
#   for path in paths:
#     with open(path,"r") as f:
#       module_txt=f.read()
    
#     for i in re.findall(r"(\w+) (\w+) \((.*)\);",module_txt)[1:]:
#       tmpi=re.findall(r'\.(\S+)\( ?[^\(\),]+\)',i[2])
#       port="("
#       for k in tmpi[:-1]:
#         port+=f".{k}({{{k}}}), "
#       port+=f".{tmpi[-1]}({{{tmpi[-1]}}})"+");"
#       if(i[0] not in items):
#         items[i[0]]=[port]
#       elif(items[i[0]]!=[port]):
#         raise Exception("Items have mismatching initiation")
#   return items

In [ ]:
# items=analyze_init(paths)

In [ ]:

# with open("input_files/ASSURE_LOCKED/modulefiles.v","r") as f:
#   module_txt=f.read()


# with open("tmp/tmpmod.v","w") as f:
#   f.write(module_txt)


# # with open("tmp.v","w") as f:
# #   f.write(t)

# gate_mapping,gates,FF=verilog_parser.extract_modules_def(module_txt)

# path="input_files/ASSURE_LOCKED/design4/design4_netlist.v"
# # "input_files/test.v"
# # "input_files/ASSURE_LOCKED/design4/design4_netlist.v"




# # # verilog=tmp1()
# # verilog=tmp2()
# # with open("tmp/tmp.v","w") as f:
# #   f.write(verilog)
# # # print(verilog)

# # print("HERE")


In [ ]:
# gate_tech,sub_module,(FF_tech,Clock_pins,Reset_pins)=verilog_parser.gates_module_extraction(verilog,gate_mapping,gates,FF)